In [24]:
import pandas as pd

In [25]:
schools_gs = pd.read_csv('schools_gs.csv')
print(schools_gs.shape)
schools_gs.head()

(950, 4)


,Unnamed: 0,School,Students,District
0,0,10/10Top ratedGreatSchools’ Summary Rating pro...,808.0,Weston School District
1,1,3/10Below averageGreatSchools’ Summary Rating ...,361.0,West Hartford School District
2,2,4/10Below averageGreatSchools’ Summary Rating ...,331.0,New Haven School District
3,3,4/10Below averageGreatSchools’ Summary Rating ...,603.0,Stamford School District
4,4,7/10Above averageGreatSchools’ Summary Rating ...,484.0,Danbury School District


In [26]:
# Convert the "School" and "District" fields to string and get rid of "Unnamed: 0"
schools_gs = schools_gs.astype(
    {
        'School': str,
        'District': str
    }
).drop(
    ['Unnamed: 0'],
    axis = 1
)

schools_gs['Rating'] = schools_gs['School'].apply(
    lambda x: x.split('/10')[0]
)
schools_gs['Rating'] = pd.to_numeric(schools_gs['Rating'], errors = 'coerce')
schools_gs['Name'] = schools_gs['School'].apply(lambda x: x.split('\n')[1])

schools_gs = schools_gs.dropna(subset=['Rating'])
schools_gs = schools_gs.drop(
    ['School'],
    axis = 1
)
schools_gs.head()

,Students,District,Rating,Name
0,808.0,Weston School District,10.0,Weston High School
1,361.0,West Hartford School District,3.0,Webster Hill School
2,331.0,New Haven School District,4.0,Sound School
3,603.0,Stamford School District,4.0,Julia A. Stark School
4,484.0,Danbury School District,7.0,Hayestown Avenue School


In [27]:
district_ratings = schools_gs[['Name', 'District','Students']].merge(
    (schools_gs['Students'] * schools_gs['Rating']).rename('Weight'),
    left_index=True,
    right_index=True
)
district_ratings.head()

,Name,District,Students,Weight
0,Weston High School,Weston School District,808.0,8080.0
1,Webster Hill School,West Hartford School District,361.0,1083.0
2,Sound School,New Haven School District,331.0,1324.0
3,Julia A. Stark School,Stamford School District,603.0,2412.0
4,Hayestown Avenue School,Danbury School District,484.0,3388.0


In [28]:
district_ratings = district_ratings.groupby('District').sum()

district_ratings = district_ratings.merge(
    (district_ratings['Weight'] / district_ratings['Students']).rename('Rating').round(1),
    left_index=True,
    right_index=True
)[['Students','Rating']]

district_ratings['Students'] = district_ratings['Students'].astype(int)

district_ratings.head()

,Students,Rating
District,,
Achievement First Hartford Ac,1149,6.0
Amistad Academy District,1081,5.0
Andover School District,204,7.0
Ansonia School District,2179,3.0
Area Cooperative Educational,1337,4.1


In [29]:
districts_acs = pd.read_csv('districts_acs.csv')
print(districts_acs.shape)
districts_acs.head()

(166, 3)


,name,households,income
0,Andover School District,1216,105328
1,Ashford School District,1774,70952
2,Barkhamsted School District,1336,109688
3,Bethany School District,2018,129133
4,Bozrah School District,986,90156


In [30]:
districts = districts_acs.merge(
    district_ratings,
    how = 'inner',
    left_on = 'name',
    right_on = 'District'
).rename(
    {'Rating': 'GS Rating'},
    axis = 1
)
districts.head(5)

,name,households,income,Students,GS Rating
0,Andover School District,1216,105328,204,7.0
1,Ashford School District,1774,70952,394,4.0
2,Barkhamsted School District,1336,109688,222,8.0
3,Bethany School District,2018,129133,364,9.0
4,Bozrah School District,986,90156,202,6.0


In [31]:
districts_gs = pd.read_csv('districts_gs.csv')
districts_gs = districts_gs.sort_values(
    by = 'district'
)
districts_gs.head()

,Unnamed: 0,district,district-href,english,math,ap_math,ap_sci,ap_other,low_income_perc,students_per_teacher,teachers_3yr,teacher_salary,rev_per_student,spend_per_student,spend_instruction,students,college,ap_total,local_rev
159,159,Achievement First Hartford Ac,https://www.greatschools.org/connecticut/hartf...,53%,53%,19%,60%,NaN,100%,11.0,83%,"$54,148","$13,711","$14,055",47%,"1,149",77%,65%,11%
181,181,Amistad Academy District,https://www.greatschools.org/connecticut/new-h...,55%,50%,13%,51%,NaN,98%,14.0,73%,"$52,216","$11,585","$12,057",44%,"1,081",91%,60%,2%
132,132,Andover School District,https://www.greatschools.org/connecticut/andov...,71%,71%,NaN,NaN,NaN,20%,8.0,100%,NaN,"$20,912","$19,848",66%,204,71%,NaN,38%
147,147,Ansonia School District,https://www.greatschools.org/connecticut/anson...,37%,30%,6%,12%,10%,66%,13.0,87%,"$44,882","$19,738","$15,647",54%,"2,283",48%,20%,39%
187,187,Area Cooperative Educational,https://www.greatschools.org/connecticut/north...,42%,32%,<1%,<1%,<1%,39%,10.0,100%,"$69,796",NaN,NaN,NaN,"1,942",32%,<1%,NaN


In [32]:
districts_gs = districts_gs[
    [
        'district',
        'english',
        'math',
        'low_income_perc',
        'students_per_teacher',
        'teachers_3yr',
        'teacher_salary',
        'rev_per_student',
        'spend_per_student',
        'spend_instruction'
     ]
]
districts_gs.head()

,district,english,math,low_income_perc,students_per_teacher,teachers_3yr,teacher_salary,rev_per_student,spend_per_student,spend_instruction
159,Achievement First Hartford Ac,53%,53%,100%,11.0,83%,"$54,148","$13,711","$14,055",47%
181,Amistad Academy District,55%,50%,98%,14.0,73%,"$52,216","$11,585","$12,057",44%
132,Andover School District,71%,71%,20%,8.0,100%,NaN,"$20,912","$19,848",66%
147,Ansonia School District,37%,30%,66%,13.0,87%,"$44,882","$19,738","$15,647",54%
187,Area Cooperative Educational,42%,32%,39%,10.0,100%,"$69,796",NaN,NaN,NaN


In [33]:
districts = pd.merge(
    districts,
    districts_gs,
    how = 'inner',
    left_on = 'name',
    right_on = 'district'
).drop(
    ['district'],
    axis = 1
)
districts

,name,households,income,Students,GS Rating,english,math,low_income_perc,students_per_teacher,teachers_3yr,teacher_salary,rev_per_student,spend_per_student,spend_instruction
0,Andover School District,1216,105328,204,7.0,71%,71%,20%,8.0,100%,NaN,"$20,912","$19,848",66%
1,Ashford School District,1774,70952,394,4.0,43%,43%,31%,9.0,98%,NaN,"$22,317","$20,894",59%
2,Barkhamsted School District,1336,109688,222,8.0,58%,58%,NaN,12.0,100%,NaN,"$22,318","$17,815",51%
3,Bethany School District,2018,129133,364,9.0,66%,66%,9%,10.0,92%,"$47,335","$21,381","$19,815",67%
4,Bozrah School District,986,90156,202,6.0,42%,42%,28%,9.0,96%,"$70,868","$31,507","$22,587",46%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,Winchester School District,4550,68750,299,6.0,49%,54%,56%,10.0,93%,"$60,547","$43,074","$24,824",33%
149,Windham School District,8590,47481,3042,3.5,37%,32%,76%,10.0,78%,"$59,975","$22,672","$20,668",54%
150,Windsor School District,10770,92199,2519,4.7,44%,44%,36%,10.0,63%,"$63,636","$26,186","$23,338",51%
151,Windsor Locks School District,5307,70067,1078,4.8,49%,41%,41%,9.0,96%,NaN,"$24,560","$23,148",59%


In [35]:
def to_percent(series):
    return pd.to_numeric(series.str.rstrip('%'), errors = 'coerce') / 100
def remove_dollar_sign(series):
    return pd.to_numeric(series.str.lstrip('$').str.replace(',',''), errors = 'coerce')

In [36]:
percentage_cols = ['english','math','low_income_perc','spend_instruction','teachers_3yr']
dollar_cols = ['teacher_salary','rev_per_student',	'spend_per_student']

districts[percentage_cols] = districts[percentage_cols].apply(to_percent)
districts[dollar_cols] = districts[dollar_cols].apply(remove_dollar_sign)

In [37]:
districts.sort_values('GS Rating', ascending = False)

,name,households,income,Students,GS Rating,english,math,low_income_perc,students_per_teacher,teachers_3yr,teacher_salary,rev_per_student,spend_per_student,spend_instruction
31,Woodbridge School District,2897,157610,830,10.0,0.74,0.74,0.07,10.0,1.00,NaN,25328,19994,0.54
12,Essex School District,3074,89261,343,10.0,0.72,0.72,0.13,9.0,1.00,70336.0,24926,20909,0.53
92,New Canaan School District,7116,190227,4220,9.7,0.90,0.88,0.00,13.0,0.94,103728.0,25888,22449,0.52
145,Westport School District,9916,206466,5494,9.5,0.83,0.83,0.04,13.0,0.97,83856.0,23670,22692,0.59
58,Darien School District,6895,232523,4789,9.5,0.85,0.85,0.02,15.0,0.90,97717.0,23819,20744,0.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,Hartford School District,46690,36278,18090,2.8,0.25,0.18,0.72,13.0,0.84,81578.0,25257,20118,0.45
95,New London School District,10732,46298,3650,2.8,0.32,0.23,0.81,14.0,0.83,55056.0,23044,20182,0.44
48,Bridgeport School District,50638,46662,19971,2.7,0.27,0.16,0.57,15.0,0.73,64000.0,22392,15915,NaN
91,New Britain School District,28232,46499,9426,1.6,0.20,0.11,0.75,14.0,0.80,59176.0,18528,16601,0.59
